In [1]:
!pip install geopandas


In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json


In [8]:
from bokeh.io import output_notebook, show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, Slider, HoverTool
from bokeh.palettes import brewer
from bokeh.layouts import widgetbox, row, column

In [9]:
shapefile = 'data/ne_110m_admin_0_countries.shp'

#Read shapefile using Geopandas
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]

#Rename columns
gdf.columns = ['country', 'country_code', 'geometry']
gdf.head()

,country,country_code,geometry
0,Fiji,FJI,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,United Republic of Tanzania,TZA,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,Western Sahara,SAH,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Canada,CAN,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,United States of America,USA,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [10]:
print(gdf[gdf['country'] == 'Antarctica'])



        country country_code  \
159  Antarctica          ATA   

                                              geometry  
159  MULTIPOLYGON (((-48.66062 -78.04702, -48.15140...  


In [11]:
#Drop row corresponding to 'Antarctica'
gdf = gdf.drop(gdf.index[159])

In [12]:
#Read meat consumption csv file
meatfile = 'data/meat-supply-per-person.csv'

meat_df = pd.read_csv(meatfile, names = ['Entity', 'Code', 'Year', 'Meat'], skiprows = 1)

meat_df.head()

,Entity,Code,Year,Meat
0,Afghanistan,AFG,1961,14.45
1,Afghanistan,AFG,1962,14.46
2,Afghanistan,AFG,1963,14.88
3,Afghanistan,AFG,1964,15.07
4,Afghanistan,AFG,1965,15.38


In [13]:
meat_df.info()
meat_df[meat_df['Code'].isnull()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11028 entries, 0 to 11027
Data columns (total 4 columns):
Entity    11028 non-null object
Code      9294 non-null object
Year      11028 non-null int64
Meat      11028 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 344.8+ KB


,Entity,Code,Year,Meat
57,Africa,NaN,1961,13.44
58,Africa,NaN,1962,13.12
59,Africa,NaN,1963,12.97
60,Africa,NaN,1964,13.07
61,Africa,NaN,1965,13.18
...,...,...,...,...
10768,Western Asia,NaN,2009,34.72
10769,Western Asia,NaN,2010,36.88
10770,Western Asia,NaN,2011,38.19
10771,Western Asia,NaN,2012,38.74


In [14]:
meat_df.info()
meat_df[meat_df['Meat'].isnull()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11028 entries, 0 to 11027
Data columns (total 4 columns):
Entity    11028 non-null object
Code      9294 non-null object
Year      11028 non-null int64
Meat      11028 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 344.8+ KB


,Entity,Code,Year,Meat


In [15]:
#Find max value of meat consumption for map
meat_df['Meat'].max()

157.56

In [16]:
# #Create test static map
# #Filter data for year 2016
# meat_df_2016 = meat_df[meat_df['Year'] == 2016]

# #Left Merge/left outer join dataframes gdf and meat_df_2016 to preserve every for in gdf
# merged = gdf.merge(meat_df_2016, left_on = 'country_code', right_on = 'Code', how = 'left')

# #Replace NaN values to string 'No data'
# #merged.fillna('No data', inplace = True)


In [17]:
#Define function that returns json_data for year selected by user
def json_data(selectedYear):
    yr = selectedYear
    df_yr = meat_df[meat_df['Year'] == yr]
    merged = gdf.merge(df_yr, left_on = 'country_code', right_on = 'Code', how = 'left')
    #merged.fillna('No data', inplace = True)
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    return json_data

In [18]:
#Read data to json
#merged_json = json.loads(merged.to_json())

#Convert to String like object
#json_data = json.dumps(merged_json)


In [19]:
#Input GeoJson source that contains features for plotting
geosource = GeoJSONDataSource(geojson = json_data(2016))

In [21]:
#Define a sequential multi-hue color palette
palette = brewer['BuPu'][8]

#Reverse color order so that dark red is highest meat consumption
palette = palette[::-1]

#Set LinearColorMapper to linearly map numbers in a range into a sequence of colors
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 160, nan_color = '#d9d9d9')

#Define custom tick labels for color bar
tick_labels = {'0': '0 kg', '10': '10 kg', '20': '20 kg', '30': '30 kg', '40': '40 kg', '50': '50 kg',
              '60': '60 kg', '70': '70 kg', '80': '80 kg', '90': '90 kg', '100': '100 kg', 
              '110': '110 kg', '120': '120 kg', '130': '130 kg', '140': '140 kg', '150': '150 kg', '160': '160 kg'}

#Hover tool
hover = HoverTool(tooltips = [('Country/region', '@country'),('Meat kg', '@Meat')])

#Create color bar
color_bar = ColorBar(color_mapper = color_mapper,
                    label_standoff = 8, width = 500, height = 20,
                    border_line_color = None,
                    location = (0, 0),
                    orientation = 'horizontal',
                    major_label_overrides = tick_labels)

#Create figure object
p = figure(plot_height = 600, plot_width = 950, toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure
p.patches('xs', 'ys', source = geosource, 
          fill_color = {'field':'Meat','transform': color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify layout
p.add_layout(color_bar, 'below')

#Define the callback function: update_plot

def update_plot(attr, old, new):
    yr = slider.value
    new_data = json_data(yr)
    geosource.geojson = new_data
    p.title.text = 'Meat Consumption kg/person/year), %d' %yr
    

#make a column layout of slider and plot
layout = column(p)
curdoc().add_root(layout)

#Display figure in Jupyter Notebook
#output_notebook()

#show(layout)

output_file("templates/meat_world.html")
show(layout)